In [87]:
#Step 1: Scraping Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.by import By

#Step 2: Database Dependencies
import pymongo

#Python Dependencies
import pandas as pd
import time


# STEP 1: Web Scraping

1. Use Splinter to create a browser object connected to NASA Mars website
2. Use BeautifulSoup to parse HTML and retrieve information

In [88]:
#define path to chrome driver
executable_path = {'executable_path': '/chromedriver'}

#Set default behaviors
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")

#Create browser object
browser = Browser('chrome', **executable_path, headless=False)

In [89]:
mars_dict = {}

### NASA Mars News

In [90]:
#Connect to NASA Mars webpage
browser.visit("https://mars.nasa.gov/news")

In [91]:
#Create a soup object 
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [92]:
#Access the titles and article teasers for each news article
articles = soup.findAll("li", class_="slide")
titles = []
teasers = []

for article in articles:
    title = article.find('div', class_="content_title").text
    titles.append(title)
    teaser = article.find('div', class_="article_teaser_body").text
    teasers.append(teaser)

#Append to the Mars Dictionary
mars_dict.update({"titles": titles, "teasers": teasers})

### JPL Mars Space Images

In [93]:
#Connect to JPL NASA web page
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")

In [94]:
#Create a soup object 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [95]:
#Use Splinter to Click on full JPG image for featured image
browser.find_by_id('full_image').first.click()

In [96]:
pic = browser.links.find_by_partial_text("more info")
pic.click()

In [97]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

img_url = soup.select_one("figure.lede a img").get("src")
feat_img_url = "https://www.jpl.nasa.gov" + img_url
print(feat_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19083_hires.jpg


In [98]:
#Update dictionary with featured image url
mars_dict.update({"Featured_Image_URL": feat_img_url})

### Mars Facts
1. Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
2. Use Pandas to convert the data to a HTML table string.

In [99]:
url = 'https://space-facts.com/mars'

In [100]:
tables = pd.read_html(url)

In [101]:
#Rename Columns
df = tables[0]
df.columns = ['Mars Planet Profile', 'Value']

In [104]:
#Convert table to an html table
mars_table = df.to_html()

#Clean up table
html_table.replace('\n', '')

#Save directly to file
df.to_html('table.html')

In [105]:
    mars_dict.update({"Mars_Facts_Table": mars_table})
    time.sleep(2)

### Mars Hemispheres

In [106]:
# Mars hemisphere name and image to be scraped
main_url = 'https://astrogeology.usgs.gov'
hemis_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
time.sleep(2)

In [107]:
#Create Soup Object
browser.visit(hemis_url)

hemis_html = browser.html
hemis_soup = BeautifulSoup(hemis_html, 'lxml')

In [108]:
# Mars Hemispheres Data Page Links
all_hemis = hemis_soup.find('div', class_='collapsible results')
mars_hemis = all_hemis.find_all('div', class_='item')

hemi_img_urls = []

# Loop through the main hemisphere data page
for hemi in mars_hemis:
    # Collect Title
    hemisphere = hemi.find('div', class_="description")
    title = hemisphere.h3.text
    
    # Scrape image links on hemisphere page
    hemi_link = hemi.a["href"]    
    browser.visit(main_url + hemi_link)
    
    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    # Create Dictionary to with key: title and value: url info
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = image_url
    
    hemi_img_urls.append(image_dict)

mars_dict.update({"image_dict": image_dict})

# Step2: MongoDB and Flask Application

In [109]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [113]:
#Create a MongoDB database
db = client.mars_db

#Add collection
data_collection = db["mars"]

#Insert dictionary into mars_db
data_collection.insert_one(mars_dict)

DuplicateKeyError: E11000 duplicate key error collection: mars_db.mars index: _id_ dup key: { _id: ObjectId('5f63b945acc00fd032c0b727') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('5f63b945acc00fd032c0b727')}, 'errmsg': "E11000 duplicate key error collection: mars_db.mars index: _id_ dup key: { _id: ObjectId('5f63b945acc00fd032c0b727') }"}